In [ ]:
import sagemaker

print(sagemaker.__version__)

session = sagemaker.Session()

In [ ]:
%%sh
# https://s3.amazonaws.com/amazon-reviews-pds/readme.html
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz /tmp

In [ ]:
prefix = 'amazon-reviews-camera'

input_data = session.upload_data(path='/tmp/amazon_reviews_us_Camera_v1_00.tsv.gz', key_prefix=prefix)

In [ ]:
from sagemaker.sklearn.processing import SKLearnProcessor

sklearn_processor = SKLearnProcessor(
    framework_version='0.23-1',
    role=sagemaker.get_execution_role(),
    instance_type='ml.c5.9xlarge',
    instance_count=1)

In [ ]:
%%time

from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor.run(
    code='preprocessing.py',
    
    inputs=[
        ProcessingInput(
                source=input_data,
                destination='/opt/ml/processing/input')
    ],
    
    outputs=[
        ProcessingOutput(
                output_name='train_data',
                source='/opt/ml/processing/train'),
        ProcessingOutput(
                output_name='validation_data',
                source='/opt/ml/processing/validation')
    ],
    
    arguments=[
               '--filename', 'amazon_reviews_us_Camera_v1_00.tsv.gz',
               '--split-ratio', '0.05',
    ]
)

In [ ]:
preprocessing_job_description = sklearn_processor.jobs[-1].describe()

output_config = preprocessing_job_description['ProcessingOutputConfig']
for output in output_config['Outputs']:
    print(output['S3Output']['S3Uri'])